<a href="https://colab.research.google.com/github/zahra-zarrabi/emotion_detection/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd "/content/drive/MyDrive/Emotion-detection"

/content/drive/.shortcut-targets-by-id/1u36KYGMcr7PbW3DSjuZPv6qQTGBnkAfX/Emotion-detection


In [ ]:

!gdown https://drive.google.com/uc?id=1X60B-uR3NtqPd4oosdotpbDgy8KOfUdr

In [ ]:
!unzip -d "src/" "data.zip"

In [ ]:
%cd 'src/'

/content/drive/.shortcut-targets-by-id/1u36KYGMcr7PbW3DSjuZPv6qQTGBnkAfX/Emotion-detection/src


In [ ]:
import numpy as np
import argparse
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from keras.callbacks import EarlyStopping
import keras

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# plots accuracy and loss curves
def plot_model_history(model_history):
    """
    Plot Accuracy and Loss curves given the model_history
    """
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig('plot.png')
    plt.show()

# Define data generators
train_dir = 'data/train'
val_dir = 'data/test'

num_train = 28709
num_val = 7178
batch_size = 64
num_epoch = 50

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# Save checkpoints
checkpoint = keras.callbacks.ModelCheckpoint('model_em/model{epoch:08d}.h5',period=1)
early_stop = EarlyStopping(monitor='val_loss',restore_best_weights=True,patience=6,mode='min')


model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
model_info = model.fit_generator(
        train_generator,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=validation_generator,
        validation_steps=num_val // batch_size,
        callbacks=[checkpoint,early_stop])

plot_model_history(model_info)
model.save_weights('model.h5')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:105: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
448/448 [==============================] - 9736s 22s/step - loss: 1.8085 - accuracy: 0.2578 - val_loss: 1.7494 - val_accuracy: 0.3278
Epoch 2/50
448/448 [==============================] - 380s 848ms/step - loss: 1.6402 - accuracy: 0.3596 - val_loss: 1.5528 - val_accuracy: 0.4120
Epoch 3/50
448/448 [==============================] - 379s 845ms/step - loss: 1.5373 - accuracy: 0.4085 - val_loss: 1.4718 - val_accuracy: 0.4332
Epoch 4/50
448/448 [==============================] - 369s 824ms/step - loss: 1.4683 - accuracy: 0.4368 - val_loss: 1.4128 - val_accuracy: 0.4626
Epoch 5/50
448/448 [==============================] - 374s 834ms/step - loss: 1.4098 - accuracy: 0.4635 - val_loss: 1.3667 - val_accuracy: 0.4798
Epoch 6/50
448/448 [==============================] - 376s 838ms/step - loss: 1.3610 - accuracy: 0.4838 - val_loss: 1.3270 - val_accuracy: 0.4985
Epoch 7/50
448/448 [==============================] - 376s 840ms/step - loss: 1.3205 - accuracy: 0.5005 - val_loss: 1.2907 - 

In [ ]:
model.save('model_emotion.h5')